In [1]:
pip install torch torchvision

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import necessary libraries

In [3]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models


Data Preprocessing and Augmentation

In [4]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(48, scale=(0.8, 1.2)),
    transforms.RandomAffine(10, translate=(0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.5)
])

#transform = transforms.Compose([
    #transforms.Resize((48,48)),
#    transforms.ToTensor(),
#    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#])


test_transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [5]:
train_dataset = ImageFolder(root='/content/drive/MyDrive/archive/train', transform=transform)
test_dataset = ImageFolder(root='/content/drive/MyDrive/archive/test',transform=test_transform)

val_size = int(0.2 * len(train_dataset))
train_size = len(train_dataset) - val_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False,num_workers=4,pin_memory=True)

In [6]:
import time
start_time = time.time()
for i, (images, labels) in enumerate(train_loader):
    end_time = time.time()
    print(f"Time for loading batch {i}: {end_time - start_time:.3f} seconds")
    break


Time for loading batch 0: 7.126 seconds


Define the model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 3 * 3, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 7)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

Intialize the model, loss and optimizer

**Cross entropy loss and SGD Optimizer**

In [ ]:
model = VGGNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)

Training

In [ ]:
#model = model.to('cuda')
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
           # val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")


Epoch 1/20 completed. Training Loss: 1.6254, Accuracy: 35.34%. Validation Loss: 1.4099, Accuracy: 46.12%
Epoch 2/20 completed. Training Loss: 1.3267, Accuracy: 49.27%. Validation Loss: 1.2466, Accuracy: 51.47%
Epoch 3/20 completed. Training Loss: 1.1755, Accuracy: 55.46%. Validation Loss: 1.1468, Accuracy: 55.81%
Epoch 4/20 completed. Training Loss: 1.0567, Accuracy: 60.14%. Validation Loss: 1.1166, Accuracy: 57.39%
Epoch 5/20 completed. Training Loss: 0.9441, Accuracy: 64.44%. Validation Loss: 1.0724, Accuracy: 59.92%
Epoch 6/20 completed. Training Loss: 0.8311, Accuracy: 69.09%. Validation Loss: 1.1166, Accuracy: 59.26%
Epoch 7/20 completed. Training Loss: 0.7159, Accuracy: 73.46%. Validation Loss: 1.1218, Accuracy: 60.88%
Epoch 8/20 completed. Training Loss: 0.5975, Accuracy: 78.11%. Validation Loss: 1.1934, Accuracy: 60.32%
Epoch 9/20 completed. Training Loss: 0.4924, Accuracy: 82.47%. Validation Loss: 1.4469, Accuracy: 58.40%
Epoch 10/20 completed. Training Loss: 0.4126, Accuracy:

Testing

In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')


Accuracy on the test set: 61.40%


SGD optimizer results in testing accuracy of 61.40%

Intialize the model, loss and optimizer
**Cross entropy loss and SGD Nesterov Optimizer**

In [ ]:
model = VGGNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001, nesterov=True)

In [ ]:
#model = model.to('cuda')
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
           # val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")


Epoch 1/20 completed. Training Loss: 1.5773, Accuracy: 37.62%. Validation Loss: 1.3744, Accuracy: 47.48%
Epoch 2/20 completed. Training Loss: 1.2920, Accuracy: 50.82%. Validation Loss: 1.1928, Accuracy: 54.42%
Epoch 3/20 completed. Training Loss: 1.1383, Accuracy: 57.24%. Validation Loss: 1.1577, Accuracy: 55.77%
Epoch 4/20 completed. Training Loss: 1.0224, Accuracy: 61.63%. Validation Loss: 1.0826, Accuracy: 59.47%
Epoch 5/20 completed. Training Loss: 0.9105, Accuracy: 66.02%. Validation Loss: 1.1212, Accuracy: 58.30%
Epoch 6/20 completed. Training Loss: 0.7820, Accuracy: 70.65%. Validation Loss: 1.1358, Accuracy: 58.72%
Epoch 7/20 completed. Training Loss: 0.6741, Accuracy: 75.29%. Validation Loss: 1.2010, Accuracy: 59.15%
Epoch 8/20 completed. Training Loss: 0.5513, Accuracy: 80.09%. Validation Loss: 1.2892, Accuracy: 58.37%
Epoch 9/20 completed. Training Loss: 0.4567, Accuracy: 83.66%. Validation Loss: 1.3515, Accuracy: 58.94%
Epoch 10/20 completed. Training Loss: 0.3547, Accuracy:

In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 60.66%


SGD Nesterov optimizer results in testing accuracy of 60.66%

Intialize the model, loss and optimizer

**Cross entropy loss and ADAM Optimizer**

In [ ]:
model = VGGNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [ ]:
#model = model.to('cuda')
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            #val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")


Epoch 1/20 completed. Training Loss: 1.9426, Accuracy: 25.09%. Validation Loss: 1.6647, Accuracy: 33.50%
Epoch 2/20 completed. Training Loss: 1.7383, Accuracy: 30.82%. Validation Loss: 1.5951, Accuracy: 38.98%
Epoch 3/20 completed. Training Loss: 1.5899, Accuracy: 36.60%. Validation Loss: 1.5832, Accuracy: 38.70%
Epoch 4/20 completed. Training Loss: 1.5355, Accuracy: 38.87%. Validation Loss: 1.5238, Accuracy: 38.37%
Epoch 5/20 completed. Training Loss: 1.5264, Accuracy: 41.12%. Validation Loss: 1.3523, Accuracy: 47.33%
Epoch 6/20 completed. Training Loss: 1.3252, Accuracy: 49.80%. Validation Loss: 1.2029, Accuracy: 53.46%
Epoch 7/20 completed. Training Loss: 1.1908, Accuracy: 55.06%. Validation Loss: 1.1417, Accuracy: 55.27%
Epoch 8/20 completed. Training Loss: 1.1078, Accuracy: 58.61%. Validation Loss: 1.0892, Accuracy: 58.94%
Epoch 9/20 completed. Training Loss: 1.0371, Accuracy: 61.59%. Validation Loss: 1.1267, Accuracy: 58.32%
Epoch 10/20 completed. Training Loss: 0.9570, Accuracy:

In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 59.97%


Adam optimizer results in test accuracy of 59.97%

Intialize the model, loss and optimizer

Cross entropy loss and ADAM AMSGrad Optimizer


In [ ]:
model = VGGNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001, amsgrad=True)

In [ ]:
#model = model.to('cuda')
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            #val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")


Epoch 1/20 completed. Training Loss: 1.9862, Accuracy: 22.00%. Validation Loss: 1.8126, Accuracy: 25.97%
Epoch 2/20 completed. Training Loss: 1.7936, Accuracy: 24.97%. Validation Loss: 1.7182, Accuracy: 30.43%
Epoch 3/20 completed. Training Loss: 1.6962, Accuracy: 30.72%. Validation Loss: 1.6047, Accuracy: 33.08%
Epoch 4/20 completed. Training Loss: 1.6192, Accuracy: 33.60%. Validation Loss: 1.6348, Accuracy: 33.50%
Epoch 5/20 completed. Training Loss: 1.5928, Accuracy: 34.51%. Validation Loss: 1.5588, Accuracy: 33.18%
Epoch 6/20 completed. Training Loss: 1.5553, Accuracy: 35.91%. Validation Loss: 1.5409, Accuracy: 41.28%
Epoch 7/20 completed. Training Loss: 1.5146, Accuracy: 40.84%. Validation Loss: 1.4312, Accuracy: 43.37%
Epoch 8/20 completed. Training Loss: 1.4309, Accuracy: 44.83%. Validation Loss: 1.3154, Accuracy: 50.32%
Epoch 9/20 completed. Training Loss: 1.2735, Accuracy: 51.35%. Validation Loss: 1.2132, Accuracy: 54.35%
Epoch 10/20 completed. Training Loss: 1.1679, Accuracy:

In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 60.45%


Adam optimizer with AMSGrad results in accuracy of 60.45%

Since SGD optimizer led to the best accuracy. I tried different LR schedulers.

Step LR Scheduler with SGD Optimizer

In [ ]:
model = VGGNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
#model = model.to('cuda')
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            #val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")
    scheduler.step()

Epoch 1/20 completed. Training Loss: 1.6208, Accuracy: 35.20%. Validation Loss: 1.4118, Accuracy: 45.99%
Epoch 2/20 completed. Training Loss: 1.3223, Accuracy: 49.33%. Validation Loss: 1.2845, Accuracy: 51.02%
Epoch 3/20 completed. Training Loss: 1.1694, Accuracy: 55.88%. Validation Loss: 1.1610, Accuracy: 56.21%
Epoch 4/20 completed. Training Loss: 1.0442, Accuracy: 60.61%. Validation Loss: 1.1452, Accuracy: 57.83%
Epoch 5/20 completed. Training Loss: 0.9337, Accuracy: 65.16%. Validation Loss: 1.1219, Accuracy: 58.75%
Epoch 6/20 completed. Training Loss: 0.8172, Accuracy: 69.58%. Validation Loss: 1.1241, Accuracy: 60.04%
Epoch 7/20 completed. Training Loss: 0.7054, Accuracy: 74.19%. Validation Loss: 1.1925, Accuracy: 59.14%
Epoch 8/20 completed. Training Loss: 0.5817, Accuracy: 78.55%. Validation Loss: 1.2527, Accuracy: 58.35%
Epoch 9/20 completed. Training Loss: 0.4766, Accuracy: 82.48%. Validation Loss: 1.2802, Accuracy: 59.24%
Epoch 10/20 completed. Training Loss: 0.3761, Accuracy:

In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 62.13%


Step LR Scheduler with SGD Optimizer with Cross entropy loss gives testing accuracy of 62.13%

RLRP (Reduce Learning rate on Plateau) with SGD Optimizer

In [ ]:
model = VGGNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.75, patience=5, verbose=True)


In [ ]:
#model = model.to('cuda')
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            #val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")
    scheduler.step(val_epoch_loss)

Epoch 1/20 completed. Training Loss: 1.7403, Accuracy: 31.88%. Validation Loss: 1.5262, Accuracy: 40.67%
Epoch 2/20 completed. Training Loss: 1.4541, Accuracy: 44.15%. Validation Loss: 1.3291, Accuracy: 46.93%
Epoch 3/20 completed. Training Loss: 1.3354, Accuracy: 49.51%. Validation Loss: 1.2756, Accuracy: 50.86%
Epoch 4/20 completed. Training Loss: 1.2519, Accuracy: 53.01%. Validation Loss: 1.2258, Accuracy: 53.98%
Epoch 5/20 completed. Training Loss: 1.1709, Accuracy: 56.39%. Validation Loss: 1.1606, Accuracy: 56.07%
Epoch 6/20 completed. Training Loss: 1.1136, Accuracy: 59.63%. Validation Loss: 1.1541, Accuracy: 57.34%
Epoch 7/20 completed. Training Loss: 1.0475, Accuracy: 61.77%. Validation Loss: 1.1334, Accuracy: 57.76%
Epoch 8/20 completed. Training Loss: 0.9820, Accuracy: 64.49%. Validation Loss: 1.2236, Accuracy: 56.99%
Epoch 9/20 completed. Training Loss: 0.9112, Accuracy: 66.75%. Validation Loss: 1.0716, Accuracy: 60.62%
Epoch 10/20 completed. Training Loss: 0.7917, Accuracy:

In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 62.32%


SGD Optimizer with RLRP scheduler results in 62.32% accuracy

# MobileNetV2

Defining model with 7 bottleneck layers


In [7]:
class Bottleneck(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels):
        super(Bottleneck, self).__init__()
        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(),
            nn.Conv2d(mid_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(),
            nn.Conv2d(mid_channels, out_channels, kernel_size=1),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        return F.relu(self.bottleneck(x) + x)

class MobileNet(nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()
        self.initial_conv = nn.Conv2d(3, 64, kernel_size=3, padding=1)

        self.bottlenecks = nn.Sequential(
            Bottleneck(64, 32, 64),
            Bottleneck(64, 32, 64),
            Bottleneck(64, 32, 64),
            Bottleneck(64, 32, 64),
            Bottleneck(64, 32, 64),
            Bottleneck(64, 32, 64),
            Bottleneck(64, 32, 64)
        )

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(64, 7)
        )

    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        x = self.bottlenecks(x)
        x = self.classifier(x)
        return x


In [8]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)  # Prevents nans when probability 0
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

SGD Optimizer with Constant learning rate and focal loss

In [28]:
model = MobileNet()
#criterion = nn.CrossEntropyLoss()
criterion = FocalLoss(alpha=1, gamma=2, reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

In [17]:
#model = model.to('cuda')
num_epochs = 40

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
           # val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")


Epoch 1/40 completed. Training Loss: 1.2456, Accuracy: 23.87%. Validation Loss: 1.2589, Accuracy: 24.92%
 Epoch 2/40 completed. Training Loss: 1.2224, Accuracy: 25.31%. Validation Loss: 1.2407, Accuracy: 25.76%
 Epoch 3/40 completed. Training Loss: 1.2003, Accuracy: 26.78%. Validation Loss: 1.2234, Accuracy: 26.61%
 Epoch 4/40 completed. Training Loss: 1.1792, Accuracy: 28.27%. Validation Loss: 1.2069, Accuracy: 27.47%
 Epoch 5/40 completed. Training Loss: 1.1590, Accuracy: 29.75%. Validation Loss: 1.1911, Accuracy: 28.34%
 Epoch 6/40 completed. Training Loss: 1.1396, Accuracy: 31.24%. Validation Loss: 1.1761, Accuracy: 29.22%
 Epoch 7/40 completed. Training Loss: 1.1209, Accuracy: 32.73%. Validation Loss: 1.1618, Accuracy: 30.11%
 Epoch 8/40 completed. Training Loss: 1.1029, Accuracy: 34.22%. Validation Loss: 1.1482, Accuracy: 31.01%
 Epoch 9/40 completed. Training Loss: 1.0856, Accuracy: 35.71%. Validation Loss: 1.1351, Accuracy: 31.91%
 Epoch 10/40 completed. Training Loss: 1.0689, 

In [16]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 59.19%


SGD Nesterov with constant learning rate and Focal Loss


In [9]:
model = MobileNet()
criterion = FocalLoss(alpha=1, gamma=2, reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001, nesterov=True)

In [14]:
#model = model.to('cuda')
num_epochs = 40

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
           # val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")


Epoch 1/40 completed. Training Loss: 1.2664, Accuracy: 24.23%. Validation Loss: 1.2774, Accuracy: 25.34%
 Epoch 2/40 completed. Training Loss: 1.2400, Accuracy: 25.47%. Validation Loss: 1.2600, Accuracy: 26.15%
 Epoch 3/40 completed. Training Loss: 1.2150, Accuracy: 26.85%. Validation Loss: 1.2435, Accuracy: 27.01%
 Epoch 4/40 completed. Training Loss: 1.1905, Accuracy: 28.20%. Validation Loss: 1.2275, Accuracy: 27.89%
 Epoch 5/40 completed. Training Loss: 1.1670, Accuracy: 29.63%. Validation Loss: 1.2120, Accuracy: 28.75%
 Epoch 6/40 completed. Training Loss: 1.1440, Accuracy: 31.08%. Validation Loss: 1.1970, Accuracy: 29.62%
 Epoch 7/40 completed. Training Loss: 1.1215, Accuracy: 32.55%. Validation Loss: 1.1825, Accuracy: 30.48%
 Epoch 8/40 completed. Training Loss: 1.0995, Accuracy: 34.03%. Validation Loss: 1.1685, Accuracy: 31.35%
 Epoch 9/40 completed. Training Loss: 1.0780, Accuracy: 35.52%. Validation Loss: 1.1548, Accuracy: 32.21%
 Epoch 10/40 completed. Training Loss: 1.0570, 

In [15]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 60.93%


Adam optimizer with constant learning rate and Focal loss

In [12]:
model = MobileNet()
criterion = FocalLoss(alpha=1, gamma=2, reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)

In [18]:
#model = model.to('cuda')
num_epochs = 40

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
           # val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")


Epoch 1/40 completed. Training Loss: 1.2664, Accuracy: 24.23%. Validation Loss: 1.2774, Accuracy: 25.34%
Epoch 2/40 completed. Training Loss: 1.2418, Accuracy: 25.92%. Validation Loss: 1.2617, Accuracy: 26.49%
 Epoch 3/40 completed. Training Loss: 1.2173, Accuracy: 27.61%. Validation Loss: 1.2462, Accuracy: 27.65%
 Epoch 4/40 completed. Training Loss: 1.1930, Accuracy: 29.28%. Validation Loss: 1.2308, Accuracy: 28.81%
 Epoch 5/40 completed. Training Loss: 1.1688, Accuracy: 30.96%. Validation Loss: 1.2156, Accuracy: 29.94%
 Epoch 6/40 completed. Training Loss: 1.1449, Accuracy: 32.64%. Validation Loss: 1.2007, Accuracy: 31.10%
 Epoch 7/40 completed. Training Loss: 1.1211, Accuracy: 34.32%. Validation Loss: 1.1860, Accuracy: 32.25%
 Epoch 8/40 completed. Training Loss: 1.0976, Accuracy: 36.02%. Validation Loss: 1.1715, Accuracy: 33.42%
 Epoch 9/40 completed. Training Loss: 1.0743, Accuracy: 37.68%. Validation Loss: 1.1572, Accuracy: 34.55%
 Epoch 10/40 completed. Training Loss: 1.0512, A

In [20]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 61.67%


Adam AMSGrad with constant learning rate and  Focal loss

In [ ]:
model = MobileNet()
criterion = FocalLoss(alpha=1, gamma=2, reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001, amsgrad=True)

In [21]:
#model = model.to('cuda')
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
           # val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")

Epoch 1/40 completed. Training Loss: 1.2543, Accuracy: 24.56%. Validation Loss: 1.2678, Accuracy: 25.62%
 Epoch 2/40 completed. Training Loss: 1.2331, Accuracy: 25.84%. Validation Loss: 1.2521, Accuracy: 26.67%
 Epoch 3/40 completed. Training Loss: 1.2125, Accuracy: 27.12%. Validation Loss: 1.2372, Accuracy: 27.73%
 Epoch 4/40 completed. Training Loss: 1.1926, Accuracy: 28.40%. Validation Loss: 1.2230, Accuracy: 28.79%
 Epoch 5/40 completed. Training Loss: 1.1734, Accuracy: 29.68%. Validation Loss: 1.2095, Accuracy: 29.85%
 Epoch 6/40 completed. Training Loss: 1.1549, Accuracy: 30.96%. Validation Loss: 1.1967, Accuracy: 30.91%
 Epoch 7/40 completed. Training Loss: 1.1371, Accuracy: 32.24%. Validation Loss: 1.1845, Accuracy: 31.97%
 Epoch 8/40 completed. Training Loss: 1.1199, Accuracy: 33.52%. Validation Loss: 1.1729, Accuracy: 33.03%
 Epoch 9/40 completed. Training Loss: 1.1033, Accuracy: 34.80%. Validation Loss: 1.1619, Accuracy: 34.09%
 Epoch 10/40 completed. Training Loss: 1.0873, 

In [22]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 60.21%


Step LR with Adam optimizer and focal loss

In [25]:
model = MobileNet()
criterion = FocalLoss(alpha=1, gamma=2, reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [27]:
#model = model.to('cuda')
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            #val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")
    scheduler.step()

Epoch 1/40 completed. Training Loss: 1.2543, Accuracy: 24.56%. Validation Loss: 1.2678, Accuracy: 25.62%
 Epoch 2/40 completed. Training Loss: 1.2331, Accuracy: 25.84%. Validation Loss: 1.2521, Accuracy: 26.67%
 Epoch 3/40 completed. Training Loss: 1.2125, Accuracy: 27.12%. Validation Loss: 1.2372, Accuracy: 27.73%
 Epoch 4/40 completed. Training Loss: 1.1926, Accuracy: 28.40%. Validation Loss: 1.2230, Accuracy: 28.79%
 Epoch 5/40 completed. Training Loss: 1.1734, Accuracy: 29.68%. Validation Loss: 1.2095, Accuracy: 29.85%
 Epoch 6/40 completed. Training Loss: 1.1549, Accuracy: 30.96%. Validation Loss: 1.1967, Accuracy: 30.91%
 Epoch 7/40 completed. Training Loss: 1.1371, Accuracy: 32.24%. Validation Loss: 1.1845, Accuracy: 31.97%
 Epoch 8/40 completed. Training Loss: 1.1199, Accuracy: 33.52%. Validation Loss: 1.1729, Accuracy: 33.03%
 Epoch 9/40 completed. Training Loss: 1.1033, Accuracy: 34.80%. Validation Loss: 1.1619, Accuracy: 34.09%
 Epoch 10/40 completed. Training Loss: 1.0873, 

In [28]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 60.27%


RLRP scheduler with Adam optimizer and focal loss



In [29]:
model = MobileNet()
criterion = FocalLoss(alpha=1, gamma=2, reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.75, patience=5, verbose=True)

In [31]:
#model = model.to('cuda')
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Training
    for images, labels in train_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    #Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            #val_images, val_labels = val_images.to('cuda'), val_labels.to('cuda')
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_accuracy = 100 * val_correct / val_total
    print(f"Epoch {epoch+1}/{num_epochs} completed. "
          f"Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%. "
          f"Validation Loss: {val_epoch_loss:.4f}, Accuracy: {val_epoch_accuracy:.2f}%")
    scheduler.step(val_epoch_loss)

Epoch 1/40 completed. Training Loss: 1.3102, Accuracy: 20.44%. Validation Loss: 1.3225, Accuracy: 21.58%
 Epoch 2/40 completed. Training Loss: 1.2887, Accuracy: 21.78%. Validation Loss: 1.3061, Accuracy: 22.73%
 Epoch 3/40 completed. Training Loss: 1.2679, Accuracy: 23.12%. Validation Loss: 1.2903, Accuracy: 23.88%
 Epoch 4/40 completed. Training Loss: 1.2476, Accuracy: 24.46%. Validation Loss: 1.2750, Accuracy: 25.03%
 Epoch 5/40 completed. Training Loss: 1.2279, Accuracy: 25.80%. Validation Loss: 1.2603, Accuracy: 26.18%
 Epoch 6/40 completed. Training Loss: 1.2087, Accuracy: 27.14%. Validation Loss: 1.2461, Accuracy: 27.33%
 Epoch 7/40 completed. Training Loss: 1.1900, Accuracy: 28.48%. Validation Loss: 1.2325, Accuracy: 28.48%
 Epoch 8/40 completed. Training Loss: 1.1718, Accuracy: 29.82%. Validation Loss: 1.2194, Accuracy: 29.63%
 Epoch 9/40 completed. Training Loss: 1.1542, Accuracy: 31.16%. Validation Loss: 1.2069, Accuracy: 30.78%
 Epoch 10/40 completed. Training Loss: 1.1371, 

In [34]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #images = images.to('cuda')
        #labels = labels.to('cuda')

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')

Accuracy on the test set: 60.91%
